In [1]:
from itertools import chain
from src_draft.utils import LOW_IMP_FEATURES, ZERO_IMP_FEATURES
import shared.ml_config_core as ml_config_core
import pandas as pd
from shared.ml_config_core import ModelConfigsCollection
from shared.ml_config_runner import run_tuning_for_configs_collection
from shared.definitions import TuningResult

from Draft import feature_builder_v2

import importlib

VERBOSE = True
importlib.reload(feature_builder_v2)
features_matrix = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True,
                                                                        ds_type=feature_builder_v2.DatasetType.BASE)
display("Distribution of Samples")
value_counts = features_matrix["TARGET"].value_counts()
df_value_counts = pd.DataFrame(
    {"Total Rows": value_counts, "Proportion": value_counts / len(features_matrix)}
)
df_value_counts.round(2)
transformers = [
    # No feature engineering tuning for for rejected/accepted df
    # ml_config_core.AcceptedRejectedDropAllButEmployment,  # Only used for testing
]

baseline_only_credit_ratings_config = {
    # Only uses  ExtScore1/2/3
    "Baseline_Only_CreditRatings": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneAUC(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "ds_type": feature_builder_v2.DatasetType.BASE_ONLY_CREDIT_RATINGS
        })
}

baseline_only_base_feats = {
    # Only includes base table (i.e. no aggregations from client history)
    "LGBM_AUC_Base_Features": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneAUC(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "ds_type": feature_builder_v2.DatasetType.BASE,
        }),
}

weighted_log_loss_config = {
    # Tuned on score combined from logloss and F1 score
    "LGBM_Weighted_LogLoss": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneWeightedLogLossF1(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "drop_cols_post_proc": ZERO_IMP_FEATURES + LOW_IMP_FEATURES
        }),
}

tune_auc_config: ModelConfigsCollection = {
    "LGBM_AUC": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneAUC(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "ds_type": feature_builder_v2.DatasetType.FULL,
            "drop_cols_post_proc": ZERO_IMP_FEATURES + LOW_IMP_FEATURES
        }),
}

dart_tune_auc_config: ModelConfigsCollection = {
    "LGBM_Dart_AUC_NEW": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMDartTuneAUC(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "ds_type": feature_builder_v2.DatasetType.FULL,
            "drop_cols_post_proc": ZERO_IMP_FEATURES + LOW_IMP_FEATURES
        }),
}

tune_auc_all_features_config: ModelConfigsCollection = {
    "LGBM_AUC_All_Features": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneAUC(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "ds_type": feature_builder_v2.DatasetType.FULL,
            # "drop_cols_post_proc": ZERO_IMP_FEATURES + LOW_IMP_FEATURES
        }),
}

configs: ModelConfigsCollection = {
    # **baseline_only_credit_ratings_config,
    # **baseline_only_base_feats,
    # **weighted_log_loss_config,
    # **tune_auc_config,
    **dart_tune_auc_config,
    # **tune_auc_all_features_config
}

Full DS size: 307511


'Distribution of Samples'

In [2]:
configs.keys()

dict_keys(['LGBM_Dart_AUC_NEW'])

In [ ]:
tuning_results = run_tuning_for_configs_collection(configs, feature_builder_v2.load_datasets_and_prepare_features)

C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
 

Appending previous history


[I 2024-04-30 14:10:40,243] A new study created in memory with name: no-name-4805c4d6-1ec9-4777-906c-84869c0c80c2


drop drop_cols_post_proc: 227
after drop_cols_post_proc: 121
Full DS size: 307511
Tunning: - transformers: transformers: 0
 total options: 0
[]
search_grid:
{}

 - model: LGBM_Dart_AUC_NEW n_iters=10 with:
 {'model__class_weight': ['balanced', None], 'model__objective': ['binary'], 'model__boosting_type': ['gbdt', 'rf', 'dart'], 'model__n_estimators': Range(50, 1000, 50, int), 'model__learning_rate': Range(0.01, 0.3, 0.01, float), 'model__max_depth': Range(3, 11, 1, int), 'model__num_leaves': Range(8, 256, 8, int), 'model__min_gain_to_split': Range(0.0, 15.0, 0.5, float), 'model__min_data_in_leaf': Range(0, 3000, 100, int), 'model__lambda_l1': Range(0, 110, 5, int), 'model__lambda_l2': Range(0, 110, 5, int), 'model__bagging_fraction': Range(0.2, 1.0, 0.1, float), 'model__feature_fraction': Range(0.2, 1.0, 0.1, float), 'model__max_bin': Range(50, 500, 25, int), 'model__drop_rate': Range(0.0, 1.0, 0.025, float)}
Fold: Tuning: n_train=246008, eval_set=61503
Fold: Tuning: n_train=246009, e

C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-04-30 14:10:47,163] Trial 0 finished with value: 0.7247591221654156 and parameters: {'model__class_weight': None, 'model__objective': 'binary', 'model__boosting_type': 'rf', 'model__n_estimators': 900, 'model__learning_rate': 0.08, 'model__max_depth': 11, 'model__num_leaves': 112, 'model__min_gain_to_split': 12.5, 'model__min_data_in_leaf': 2700, 'model__lambda_l1': 50, 'model__lambda_l2': 15, 'model__bagging_fraction': 0.4, 'model__feature_fraction': 0.8, 'model__max_bin': 225, 'model__drop_rate': 0.55}. Best is trial 0 with value: 0.7247591221654156.
C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\

...

[I 2024-04-30 14:40:03,043] Trial 29 finished with value: 0.7750172313526884 and parameters: {'model__class_weight': 'balanced', 'model__objective': 'binary', 'model__boosting_type': 'dart', 'model__n_estimators': 850, 'model__learning_rate': 0.28, 'model__max_depth': 11, 'model__num_leaves': 112, 'model__min_gain_to_split': 6.0, 'model__min_data_in_leaf': 2700, 'model__lambda_l1': 55, 'model__lambda_l2': 65, 'model__bagging_fraction': 0.2, 'model__feature_fraction': 0.7, 'model__max_bin': 250, 'model__drop_rate': 0.125}. Best is trial 28 with value: 0.77509084497967.
C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\lightgbm\callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


Fold: Tuning: n_train=246008, eval_set=61503


C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\lightgbm\callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


Fold: Tuning: n_train=246009, eval_set=61502


C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\lightgbm\callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


In [ ]:
for t, v in tuning_results.items():
    display(t)
    display(v.hyper_param_all_cv_results)



In [ ]:
tuning_results.keys()

In [ ]:
# Results:

# Only main dataset:
# - top res      : 0.753
# - full + gbdt  : 0.765
# - full + dart  : 0.766


In [ ]:
from Draft.feature_builder_v2 import DatasetType

importlib.reload(feature_builder_v2)
features_matrix_full = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True,
                                                                             ds_type=DatasetType.FULL)


In [ ]:
features_matrix_full

In [ ]:
from Draft.feature_builder_v2 import DatasetType

importlib.reload(feature_builder_v2)
features_matrix_full = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True,
                                                                             ds_type=DatasetType.FULL)
features_matrix_base = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True,
                                                                             ds_type=DatasetType.BASE)


In [ ]:
features_matrix_full

In [ ]:
importlib.reload(feature_builder_v2)
features_matrix = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True,
                                                                        ds_type=feature_builder_v2.DatasetType.BASE_ONLY_CREDIT_RATINGS)
from shared import stats_utils

stats_utils.nan_summary(features_matrix)